In [3]:
import pandas as pd
#import pandavro as pda
import lxml.html as lh
from urllib.request import urlopen, Request
import requests
pd.__version__


'1.0.1'

In [4]:
url = "http://www.brewunited.com/hop_database.php"
print(url)

http://www.brewunited.com/hop_database.php


In [5]:


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(url=url, headers=headers) 
page = requests.get(url)

print(page) 
doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
print(tr_elements)


#Check the length of the first 50 rows
[len(T) for T in tr_elements[:50]]


<Response [200]>
[<Element tr at 0x19d119c2098>, <Element tr at 0x19d119e40e8>, <Element tr at 0x19d119e4098>, <Element tr at 0x19d119e4138>, <Element tr at 0x19d119e4188>, <Element tr at 0x19d119e41d8>, <Element tr at 0x19d119e4228>, <Element tr at 0x19d119e4278>, <Element tr at 0x19d119e42c8>, <Element tr at 0x19d119e4318>, <Element tr at 0x19d119e4368>, <Element tr at 0x19d119e43b8>, <Element tr at 0x19d119e4408>, <Element tr at 0x19d119e4458>, <Element tr at 0x19d119e44a8>, <Element tr at 0x19d119e44f8>, <Element tr at 0x19d119e4548>, <Element tr at 0x19d119e4598>, <Element tr at 0x19d119e45e8>, <Element tr at 0x19d119e4638>, <Element tr at 0x19d119e4688>, <Element tr at 0x19d119e46d8>, <Element tr at 0x19d119e4728>, <Element tr at 0x19d119e4778>, <Element tr at 0x19d119e47c8>, <Element tr at 0x19d119e4818>, <Element tr at 0x19d119e4868>, <Element tr at 0x19d119e48b8>, <Element tr at 0x19d119e4908>, <Element tr at 0x19d119e4958>, <Element tr at 0x19d119e49a8>, <Element tr at 0x19d1

[6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6]

In [6]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Hop"
2:"Origin"
3:"Type"
4:"Alpha"
5:"Beta"
6:"Notes"


In [7]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=6:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [8]:
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(20) 

,Hop,Origin,Type,Alpha,Beta,Notes
0,0,,,0.00,0.00,
1,007: The Golden Hop,,,0.00,0.00,
2,007: The Golden Hop,,,0.00,0.00,
3,007: The Golden Hop,,,0.00,0.00,
4,007: The Golden Hop,,,0.00,0.00,
5,Admiral,United Kingdom,Bittering,14.75,5.60,Bittering hops derived from Wye Challenger. Go...
6,Ahtanum,U.S.,Aroma,6.00,5.25,Distinctive aromatic hops with moderate bitter...
7,Amarillo,U.S.,Aroma,8.50,6.00,"Unknown origin, but character similar to Casca..."
8,Amarillo,,,0.00,0.00,
9,Amarillo,,,0.00,0.00,


In [17]:
##cleaning Duplicates and rubish
df = df.drop_duplicates()
df = df[df['Hop'] != 'Hop']
df = df[df['Hop'] != '0']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [25]:

df = df[df['Origin'] != '']
df = df[df['Alpha']!='0.00']
dfold.head(50)

,Hop,Origin,Type,Alpha,Beta,Notes
5,Admiral,United Kingdom,Bittering,14.75,5.60,Bittering hops derived from Wye Challenger. Go...
6,Ahtanum,U.S.,Aroma,6.00,5.25,Distinctive aromatic hops with moderate bitter...
7,Amarillo,U.S.,Aroma,8.50,6.00,"Unknown origin, but character similar to Casca..."
74,Apollo,U.S.,Bittering,17.00,6.75,Super high alpha variety from the Hopsteiner b...
76,Aquila,U.S.,Aroma,6.50,3.00,Aroma hops developed in 1988. Limited use due ...
77,Atlas,Slovenia,Bittering,9.00,3.70,A cross between Brewers Gold and Slovenian wil...
78,Aurora,Slovenia,Both,8.25,3.50,Also called Super Styrian. This is a hybrid be...
79,Australian Summer,Australia,Aroma,6.00,5.45,Aroma hop from Australia. It features balanced...
86,Banner,U.S.,Bittering,10.00,4.00,Bittering hops developed in 1988. Use largely ...
91,Bobek,Slovenia,Both,5.25,5.00,Also called Styrian Golding B - a cross betwee...


In [26]:
df.head(20)

,Hop,Origin,Type,Alpha,Beta,Notes
5,Admiral,United Kingdom,Bittering,14.75,5.60,Bittering hops derived from Wye Challenger. Go...
6,Ahtanum,U.S.,Aroma,6.00,5.25,Distinctive aromatic hops with moderate bitter...
7,Amarillo,U.S.,Aroma,8.50,6.00,"Unknown origin, but character similar to Casca..."
74,Apollo,U.S.,Bittering,17.00,6.75,Super high alpha variety from the Hopsteiner b...
76,Aquila,U.S.,Aroma,6.50,3.00,Aroma hops developed in 1988. Limited use due ...
77,Atlas,Slovenia,Bittering,9.00,3.70,A cross between Brewers Gold and Slovenian wil...
78,Aurora,Slovenia,Both,8.25,3.50,Also called Super Styrian. This is a hybrid be...
79,Australian Summer,Australia,Aroma,6.00,5.45,Aroma hop from Australia. It features balanced...
86,Banner,U.S.,Bittering,10.00,4.00,Bittering hops developed in 1988. Use largely ...
91,Bobek,Slovenia,Both,5.25,5.00,Also called Styrian Golding B - a cross betwee...


In [27]:
df.to_json (r'C:\Users\bceolincamar\Documents\GitHub\homebrew\Export_Hops.json', orient='records')